##### Análise de desempenhos de modelos

In [11]:
import pandas as pd
import sys
import os
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

In [12]:
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

In [13]:
from avaliador import funcao_avaliadora
from tratamento_outliers import tratar_outliers

In [14]:
df = pd.read_csv("../data/dados_transformados.csv")

In [15]:
colunas_outliers = ['bmi', 'blood_glucose_level', 'hbA1c_level']

for coluna in colunas_outliers:
    tratar_outliers(df, coluna, 'zscore')

In [16]:
X = df.drop(['diabetes'], axis=1)
y = df['diabetes']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [18]:
modelos = {
    "LogisticRegression": LogisticRegression(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
    "GradientBoostingClassifier": GradientBoostingClassifier(),
    "XGBClassifier": XGBClassifier(),
}

In [19]:
melhores_modelos = {
    'accuracy': {'nome': None, 'valor': 0},
    'precision': {'nome': None, 'valor': 0},
    'recall': {'nome': None, 'valor': 0},
    'f1': {'nome': None, 'valor': 0},
    'roc_auc': {'nome': None, 'valor': 0},
    'mcc': {'nome': None, 'valor': 0},
}

In [20]:
scalers = [
    ('RobustScaler', RobustScaler()),
    ('StandardScaler', StandardScaler()),
    ('MinMaxScaler', MinMaxScaler()),
    ('MaxAbsScaler', MaxAbsScaler())
]

In [21]:
for nome_scaler, scaler in scalers:
    for nome, modelo in modelos.items():
        pipeline = Pipeline([
            ('scaler', scaler),
            ('pca', PCA(n_components=8)),
            ('feature_selection', SelectKBest(score_func=f_classif, k='all')),
            ('modelo', modelo)
        ])

        resultados = funcao_avaliadora(pipeline, X_train, X_test, y_train, y_test)

        accuracy_test = resultados["Classification Report Test"]['accuracy']
        precision_test = resultados["Classification Report Test"]['weighted avg']['precision']
        recall_test = resultados["Classification Report Test"]['weighted avg']['recall']
        f1_test = resultados["Classification Report Test"]['weighted avg']['f1-score']
        roc_auc_test = resultados["ROC AUC Test"]
        mcc_test = resultados["Matthews Correlation Coefficient Test"]

        print(f'\n{nome} com {nome_scaler}:')
        print(f'Accuracy: {accuracy_test}')
        print(f'Precision: {precision_test}')
        print(f'Recall: {recall_test}')
        print(f'F1 Score: {f1_test}')
        print(f'ROC AUC: {roc_auc_test}')
        print(f'Matthews Correlation Coefficient: {mcc_test}\n')
        print(f'Confusion Matrix: \n{resultados["Confusion Matrix Test"]}')
        print('-' * 60)

        if accuracy_test > melhores_modelos['accuracy']['valor']:
            melhores_modelos['accuracy']['nome'] = f'{nome} com {nome_scaler}'
            melhores_modelos['accuracy']['valor'] = accuracy_test
        
        if precision_test > melhores_modelos['precision']['valor']:
            melhores_modelos['precision']['nome'] = f'{nome} com {nome_scaler}'
            melhores_modelos['precision']['valor'] = precision_test
        
        if recall_test > melhores_modelos['recall']['valor']:
            melhores_modelos['recall']['nome'] = f'{nome} com {nome_scaler}'
            melhores_modelos['recall']['valor'] = recall_test
        
        if f1_test > melhores_modelos['f1']['valor']:
            melhores_modelos['f1']['nome'] = f'{nome} com {nome_scaler}'
            melhores_modelos['f1']['valor'] = f1_test

        if roc_auc_test > melhores_modelos['roc_auc']['valor']:
            melhores_modelos['roc_auc']['nome'] = f'{nome} com {nome_scaler}'
            melhores_modelos['roc_auc']['valor'] = roc_auc_test

        if mcc_test > melhores_modelos['mcc']['valor']:
            melhores_modelos['mcc']['nome'] = f'{nome} com {nome_scaler}'
            melhores_modelos['mcc']['valor'] = mcc_test

print("\nMelhores modelos:")
print(f'Melhor modelo para Accuracy: {melhores_modelos["accuracy"]["nome"]} com valor {melhores_modelos["accuracy"]["valor"]}')
print(f'Melhor modelo para Precision: {melhores_modelos["precision"]["nome"]} com valor {melhores_modelos["precision"]["valor"]}')
print(f'Melhor modelo para Recall: {melhores_modelos["recall"]["nome"]} com valor {melhores_modelos["recall"]["valor"]}')
print(f'Melhor modelo para F1 Score: {melhores_modelos["f1"]["nome"]} com valor {melhores_modelos["f1"]["valor"]}')
print(f'Melhor modelo para ROC AUC: {melhores_modelos["roc_auc"]["nome"]} com valor {melhores_modelos["roc_auc"]["valor"]}')
print(f'Melhor modelo para MCC: {melhores_modelos["mcc"]["nome"]} com valor {melhores_modelos["mcc"]["valor"]}')


LogisticRegression com RobustScaler:
Accuracy: 0.96135
Precision: 0.9591863352529594
Recall: 0.96135
F1 Score: 0.958249540420309
ROC AUC: 0.9603390688037174
Matthews Correlation Coefficient: 0.7180508918139418

Confusion Matrix: 
[[18205   142]
 [  631  1022]]
------------------------------------------------------------

DecisionTreeClassifier com RobustScaler:
Accuracy: 0.95155
Precision: 0.9527649424972618
Recall: 0.95155
F1 Score: 0.9521135808409911
ROC AUC: 0.8521732735053041
Matthews Correlation Coefficient: 0.6882308389548502

Confusion Matrix: 
[[17819   528]
 [  441  1212]]
------------------------------------------------------------

RandomForestClassifier com RobustScaler:
Accuracy: 0.96955
Precision: 0.9692747556639668
Recall: 0.96955
F1 Score: 0.9669392571992543
ROC AUC: 0.9639154821100034
Matthews Correlation Coefficient: 0.7814696293267999

Confusion Matrix: 
[[18301    46]
 [  563  1090]]
------------------------------------------------------------

GradientBoostingClas